## run this cell before starting spark cluster connection (corrects module error)

In [1]:
# spark.jars.packages com.databricks:spark-avro_2.11:4.0.0
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_lookup = spark.read.json("/user/sprenner/lookup_final_small_encoded.json")

In [5]:
df_lookup.count()
#3694965

3694964

In [6]:
df_april = spark.read.json("/user/sprenner/training_official_april_proportion.json")

In [7]:
df_april.count()
#777286

777286

In [8]:
df_lookup.describe().toPandas()

summary                 BYTES                          CHILD_NAME  \
0  count   3694964               3694964                              
1  mean    5.1949375501406157E8  None                                 
2  stddev  1.2317919017903147E9  None                                 
3  min     10000                 AOD.05371635._000001.pool.root.1     
4  max     9999989               log.13787275._005461.job.log.tgz.1   

         CHILD_NAMEHash CHILD_SCOPE        CHILD_SCOPEHash  \
0  3694964               3694964     3694964                 
1  613624.0631261901     None        -2.9690362655605114E8   
2  1.2398551343729815E9  None        8.984975141000296E8     
3  -2147483486           data10_hi   -2057006107             
4  2147482617            mc16_valid  2109319881              

                                                                                                                 join_key_lookup  
0  3694964                                                                                                                        
1  None                                                                                                                           
2  None                                                                                                                           
3  data10_hi:data10_hi.00169927.physics_bulk.ROOT.NTUP_HI.r2111_p456_tid289065_00                                                 
4  mc16_valid:mc16_valid.423105.Pythia8EvtGen_A14NNPDF23LO_gammajet_DP280_500.merge.AOD.e5094_s3238_r10441_r10210_tid13765955_00

In [9]:
#df_april = df_april.withColumn("join_key_april", concat_ws(':', df_april["datasetScope"], df_april["dataset"]))
#df_april.createOrReplaceTempView("april")

In [10]:
df_lookup.createOrReplaceTempView("lookup")
df_april.createOrReplaceTempView("april")

In [11]:
spark.sql("SELECT * FROM lookup WHERE join_key_lookup IS NULL").toPandas()
# hier wurde ein eintrag gefunden mit überall None, die Hashes daher beide 42

Empty DataFrame
Columns: [BYTES, CHILD_NAME, CHILD_NAMEHash, CHILD_SCOPE, CHILD_SCOPEHash, join_key_lookup]
Index: []

In [12]:
df_april.printSchema()

root
 |-- account: string (nullable = true)
 |-- accountHash: long (nullable = true)
 |-- clientState: string (nullable = true)
 |-- clientStateHash: long (nullable = true)
 |-- count: long (nullable = true)
 |-- dataset: string (nullable = true)
 |-- datasetHash: long (nullable = true)
 |-- datasetScope: string (nullable = true)
 |-- datasetScopeHash: long (nullable = true)
 |-- dataset_version: string (nullable = true)
 |-- dataset_versionHash: long (nullable = true)
 |-- datatype: string (nullable = true)
 |-- datatypeHash: long (nullable = true)
 |-- eventType: string (nullable = true)
 |-- eventTypeHash: long (nullable = true)
 |-- eventVersion: string (nullable = true)
 |-- eventVersionHash: long (nullable = true)
 |-- fileScope: string (nullable = true)
 |-- filename: string (nullable = true)
 |-- filenameHash: long (nullable = true)
 |-- filesize_double: double (nullable = true)
 |-- hits: long (nullable = true)
 |-- hostname: string (nullable = true)
 |-- hostnameHash: long (n

## how many unique in april traces can be looked up

In [13]:
spark.sql("SELECT COUNT(DISTINCT join_key_lookup) FROM april WHERE join_key_lookup IS NOT NULL").toPandas()
#12361

count(DISTINCT join_key_lookup)
0  12361

## how many unique datasets are in april traces including files that belong to multiple datasets

In [14]:
spark.sql("SELECT COUNT(DISTINCT join_key_april) FROM april WHERE join_key_april IS NOT NULL").toPandas()
#12482

count(DISTINCT join_key_april)
0  12482

In [15]:
spark.sql("SELECT * FROM april WHERE join_key_lookup IS NULL").limit(50).toPandas()

account  accountHash         clientState  clientStateHash count  \
0   drafanoh -33750993     DONE               -878527175        None   
1   drafanoh -33750993     DONE               -878527175        None   
2   drafanoh -33750993     DONE               -878527175        None   
3   drafanoh -33750993     DONE               -878527175        None   
4   drafanoh -33750993     DONE               -878527175        None   
5   drafanoh -33750993     DONE               -878527175        None   
6   drafanoh -33750993     DONE               -878527175        None   
7   drafanoh -33750993     DONE               -878527175        None   
8   drafanoh -33750993     DONE               -878527175        None   
9   drafanoh -33750993     DONE               -878527175        None   
10  drafanoh -33750993     DONE               -878527175        None   
11  drafanoh -33750993     ServiceUnavailable -1394468120       None   
12  drafanoh -33750993     DONE               -878527175        None   
13  drafanoh -33750993     DONE               -878527175        None   
14  drafanoh -33750993     DONE               -878527175        None   
15  pgadow    1992700062   DONE               -878527175        None   
16  drafanoh -33750993     DONE               -878527175        None   
17  drafanoh -33750993     DONE               -878527175        None   
18  drafanoh -33750993     DONE               -878527175        None   
19  jojungge -1078508397   DONE               -878527175        None   
20  jojungge -1078508397   DONE               -878527175        None   
21  jojungge -1078508397   DONE               -878527175        None   
22  jojungge -1078508397   DONE               -878527175        None   
23  jojungge -1078508397   DONE               -878527175        None   
24  jojungge -1078508397   DONE               -878527175        None   
25  jojungge -1078508397   DONE               -878527175        None   
26  jojungge -1078508397   DONE               -878527175        None   
27  jojungge -1078508397   DONE               -878527175        None   
28  jojungge -1078508397   DONE               -878527175        None   
29  jojungge -1078508397   DONE               -878527175        None   
30  jojungge -1078508397   DONE               -878527175        None   
31  shhan     932462984    DONE               -878527175        None   
32  shhan     932462984    DONE               -878527175        None   
33  abnag     1005258035   DONE               -878527175        None   
34  ckaldero -556114008    DONE               -878527175        None   
35  luambroz -1063736139   DONE               -878527175        None   
36  hegormse -2036406190   DONE               -878527175        None   
37  sgargiul  1343070029   DONE               -878527175        None   
38  sgargiul  1343070029   DONE               -878527175        None   
39  sgargiul  1343070029   DONE               -878527175        None   
40  sgargiul  1343070029   DONE               -878527175        None   
41  sgargiul  1343070029   DONE               -878527175        None   
42  sgargiul  1343070029   DONE               -878527175        None   
43  sgargiul  1343070029   DONE               -878527175        None   
44  sgargiul  1343070029   DONE               -878527175        None   
45  sgargiul  1343070029   DONE               -878527175        None   
46  sgargiul  1343070029   DONE               -878527175        None   
47  sgargiul  1343070029   DONE               -878527175        None   
48  sgargiul  1343070029   DONE               -878527175        None   
49  sgargiul  1343070029   DONE               -878527175        None   

                                                                                                                               dataset  \
0   mc16_13TeV.410472.PhPy8EG_A14_ttbar_hdamp258p75_dil.merge.EVNT.e6348_e5984_tid13427748_00                                            
1   mc16_13TeV.410472.PhPy8EG_A14_ttbar_hdamp258p75_dil.

In [16]:
spark.sql("SELECT * FROM april WHERE join_key_lookup IS NULL AND (id_proportion IS NOT NULL OR int_percentage IS NOT NULL)").limit(10).toPandas()

Empty DataFrame
Columns: [account, accountHash, clientState, clientStateHash, count, dataset, datasetHash, datasetScope, datasetScopeHash, dataset_version, dataset_versionHash, datatype, datatypeHash, eventType, eventTypeHash, eventVersion, eventVersionHash, fileScope, filename, filenameHash, filesize_double, hits, hostname, hostnameHash, id, id_proportion, int_percentage, join_key_april, join_key_lookup, prod_step, prod_stepHash, project, projectHash, protocol, protocolHash, remoteSite, remoteSiteHash, run_number, run_numberHash, scopeHash, stream_name, stream_nameHash, traceId, traceIdHash, traceIp, traceIpHash, traceTimeentryUnix, transferStart, uuid, uuidHash]
Index: []

In [17]:
df_lookup.printSchema()

root
 |-- BYTES: string (nullable = true)
 |-- CHILD_NAME: string (nullable = true)
 |-- CHILD_NAMEHash: long (nullable = true)
 |-- CHILD_SCOPE: string (nullable = true)
 |-- CHILD_SCOPEHash: long (nullable = true)
 |-- join_key_lookup: string (nullable = true)



In [18]:
spark.sql("SELECT * FROM lookup WHERE join_key_lookup == 'mc16_13TeV:mc16_13TeV.309001.Pythia8EvtGen_A14NNPDF23LO_WmuHNL50_30G_lt1dd_lepfilt_ch4.deriv.NTUP_PILEUP.e6165_e5984_a875_r9364_r9315_p3288'")

DataFrame[BYTES: string, CHILD_NAME: string, CHILD_NAMEHash: bigint, CHILD_SCOPE: string, CHILD_SCOPEHash: bigint, join_key_lookup: string]

In [19]:
spark.sql("SELECT * FROM april WHERE join_key_april == 'mc16_13TeV:mc16_13TeV.309001.Pythia8EvtGen_A14NNPDF23LO_WmuHNL50_30G_lt1dd_lepfilt_ch4.deriv.NTUP_PILEUP.e6165_e5984_a875_r9364_r9315_p3288'").toPandas()

account  accountHash clientState  clientStateHash count  \
0  hegormse -2036406190   DONE       -878527175        None   

                                                                                                                            dataset  \
0  mc16_13TeV.309001.Pythia8EvtGen_A14NNPDF23LO_WmuHNL50_30G_lt1dd_lepfilt_ch4.deriv.NTUP_PILEUP.e6165_e5984_a875_r9364_r9315_p3288   

   datasetHash datasetScope  datasetScopeHash  \
0 -233443132    mc16_13TeV  -1447489496         

                      dataset_version  dataset_versionHash     datatype  \
0  e6165_e5984_a875_r9364_r9315_p3288  490223041            NTUP_PILEUP   

   datatypeHash eventType  eventTypeHash eventVersion  eventVersionHash  \
0  1967069222    download  59378080       1.14.11     -2044360925         

    fileScope                                  filename  filenameHash  \
0  mc16_13TeV  NTUP_PILEUP.13684268._000001.pool.root.1  1690642080     

   filesize_double  hits           hostname  hostnameHash      id  \
0  14421.0          1     lxplus020.cern.ch  1498985269    672859   

  id_proportion int_percentage  \
0  None          None            

                                                                                                                                join_key_april  \
0  mc16_13TeV:mc16_13TeV.309001.Pythia8EvtGen_A14NNPDF23LO_WmuHNL50_30G_lt1dd_lepfilt_ch4.deriv.NTUP_PILEUP.e6165_e5984_a875_r9364_r9315_p3288   

  join_key_lookup prod_step  prod_stepHash     project  projectHash protocol  \
0  None            deriv    -1058503065     mc16_13TeV -1447489496   srm       

   protocolHash    remoteSite  remoteSiteHash run_number  run_numberHash  \
0  863299160     PIC_DATADISK -419511324       309001     2045877011       

    scopeHash                                                stream_name  \
0 -1447489496  Pythia8EvtGen_A14NNPDF23LO_WmuHNL50_30G_lt1dd_lepfilt_ch4   

   stream_nameHash                           traceId  traceIdHash  \
0 -1534409593       2ac641dad71448a0b951bed4052f2d6d -835403085     

                  traceIp  traceIpHash  traceTimeentryUnix  transferStart  \
0  ::ffff:137.138.148.190 -767262405    1.524567e+09        1.524567e+09    

                               uuid    uuidHash  
0  b6d4291629b04cb0b98b1b33510ca347 -1562213315